In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import plotly.express as px

import glob

from locations import *
from parsing import *

In [4]:
df = pd.read_csv('madrid_parsed.csv')
df

,price,title,location,lat,lng,Jardín,Cocina equipada,Referencia,Clasificación,Piscina,...,Superficie solar,Calefacción,Emisiones,Teléfono,Ascensor,Chimenea,Se aceptan mascotas,Pedir más información,Interior,Soleado
0,176000.0,Apartamento en venta en Abantos-Monte Carmelo-...,Abantos-Monte Carmelo-El Rosario (San Lorenzo ...,40.593445,-4.145386,Comunitario,Amueblada a estrenar,4994318-VF-812,En trámite,Comunitaria,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,116500.0,"Apartamento en venta en Calle de Ercilla, 20, ...","Calle de Ercilla, 20, cerca de Calle del Labra...",40.402079,-3.702151,NaN,NaN,EP805-1309,En trámite,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,169000.0,Apartamento en venta en Ajalvir,Ajalvir,40.534457,-3.479415,NaN,NaN,IFC76153-1690TN,En trámite,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,174000.0,Apartamento en venta en Alcorcón,Casco Antiguo (Alcorcón),40.344358,-3.825283,NaN,NaN,SA1398-4340/6592,No indicado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,130000.0,Apartamento en venta en Alcorcón,Casco Antiguo (Alcorcón),40.347096,-3.827826,NaN,NaN,SA2135-04930/7619,No indicado,NaN,...,NaN,Eléctrica,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15832,135000.0,Piso en venta en Calle de la Circunvalación,Calle de la Circunvalación. Zona Suroeste (Tor...,40.454606,-3.455234,NaN,NaN,SA4463-BANCO-0001,No indicado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15833,210000.0,Piso en venta en Calle de Urano,Calle de Urano. Zona Suroeste (Torrejón de Ardoz),40.441928,-3.473036,NaN,NaN,SA3697-IL-04891,En trámite,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15834,324600.0,Piso en venta en Fresnos,Zona Suroeste (Torrejón de Ardoz),40.448403,-3.470899,NaN,NaN,SA555-39508999/1670,No indicado,Comunitaria,...,NaN,Gas natural,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15835,249900.0,Piso en venta en Calle de Cibeles,Calle de Cibeles. Zona Suroeste (Torrejón de A...,40.444707,-3.473520,NaN,NaN,4525678-000021,En trámite,NaN,...,NaN,Gas natural,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
